In [1]:
!pip install torch --quiet
!pip install sklearn --quiet
!pip install datasets --quiet
!apt install git-lfs --quiet
!pip install transformers --quiet
!pip install sentencepiece --quiet


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install pandas --quiet
!pip install numpy --quiet


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install beautifulsoup4 --quiet
!pip install lyricsgenius --quiet
!pip install billboard.py


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [14]:
# !pip install git+https://github.com/johnwmillr/LyricsGenius.git --quiet

  Cloning https://github.com/johnwmillr/LyricsGenius.git to /private/var/folders/zk/j144yn6s7034375p4gjwc6j00000gn/T/pip-req-build-9a1jmf8w
  Running command git clone --filter=blob:none --quiet https://github.com/johnwmillr/LyricsGenius.git /private/var/folders/zk/j144yn6s7034375p4gjwc6j00000gn/T/pip-req-build-9a1jmf8w
  Resolved https://github.com/johnwmillr/LyricsGenius.git to commit bec02665b807941ca95e045be910e861789fc4a7
  Preparing metadata (setup.py) ... done


In [4]:
!pip install python-dotenv
from dotenv import load_dotenv
import dotenv
import os
%load_ext dotenv
%dotenv


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [5]:
# Import Genious access token

In [6]:
load_dotenv(os.path.join(os.getcwd(),'.env'))
GENIOUS_ACCESS_TOKEN = os.environ.get('GENIOUS_ACCESS_TOKEN')

In [8]:
import pandas as pd
import numpy as np
import requests

# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns
from IPython.display import HTML, display
from bs4 import BeautifulSoup
# from nltk.tokenize import RegexpTokenizer
import lyricsgenius as genius
import billboard
import sys
import re
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# import string
# from nltk.stem import PorterStemmer
from datetime import datetime, date, timedelta
# from wordcloud import WordCloud
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# import spacy
# from collections import Counter
# from os import path
# from PIL import Image
# from keras.models import model_from_json
# import pickle


import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random
from tqdm.auto import tqdm, trange
from huggingface_hub import notebook_login
import os
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None
display(HTML("<style>.container { width:100% !important; }</style>"))
% config InlineBackend.figure_format = 'png'

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


UsageError: Line magic function `%` not found.


In [9]:
def disable_pandas_warnings():
    import warnings
    warnings.resetwarnings()  # Maybe somebody else is messing with the warnings system?
    warnings.filterwarnings('ignore')  # Ignore everything
    # ignore everything does not work: ignore specific messages, using regex
    warnings.filterwarnings('ignore', '.*A value is trying to be set on a copy of a slice from a DataFrame.*')
    warnings.filterwarnings('ignore', '.*indexing past lexsort depth may impact performance*')

In [10]:
disable_pandas_warnings()

In [11]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [12]:
def collect_songs_from_billboard_each_day(start_date_str, end_date_str, chart_name='hot-100', table_path = None,to_csv = False):
    start_date = datetime.strptime(
        start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    columns = ["Rank",
                    "Song Title",
                    "Artist",
                    "Date",
                    "Year"]
    dataset = pd.DataFrame(columns = columns, data = [])
    if to_csv:
        dataset.to_csv(table_path,index=False)

    sys.stdout.write(
        "\r" + "Collecting Songs from " + start_date_str + " to " + end_date_str + " via https://www.billboard.com")
    sys.stdout.flush()
    for single_date in tqdm(daterange(start_date, end_date)):
        print(single_date.strftime('%Y-%m-%d'))
        chart = billboard.ChartData(chart_name, date=single_date.strftime('%Y-%m-%d'))
        for song in chart:
            row = {
                "Rank": song.rank,
                "Song Title": song.title,
                "Artist": song.artist,
                "Date": single_date.strftime('%Y-%m-%d'),
                "Year": single_date.strftime('%Y')
            }
            if to_csv:
                pd.DataFrame([row]).to_csv(table_path, mode='a',index=False, header=False)
            else:
                dataset = dataset.append(row, ignore_index=True)  #pd.concat([dataset,pd.DataFrame(row)],ignore_index=True)#
    if not to_csv:
        return dataset


In [13]:
all_songs = collect_songs_from_billboard_each_day('2022-12-30', '2023-01-01',table_path='all_songs.csv',to_csv = True)

0it [00:00, ?it/s]

2022-12-30


1it [00:05,  5.05s/it]

2022-12-31


2it [00:09,  4.86s/it]


In [14]:
all_songs = pd.read_csv("all_songs.csv")

In [15]:
all_songs.head(70)

,Rank,Song Title,Artist,Date,Year
0,1,All I Want For Christmas Is You,Mariah Carey,2022-12-30,2022
1,2,Rockin' Around The Christmas Tree,Brenda Lee,2022-12-30,2022
2,3,Jingle Bell Rock,Bobby Helms,2022-12-30,2022
3,4,A Holly Jolly Christmas,Burl Ives,2022-12-30,2022
4,5,Last Christmas,Wham!,2022-12-30,2022
...,...,...,...,...,...
65,66,Victoria's Secret,Jax,2022-12-30,2022
66,67,Lavender Haze,Taylor Swift,2022-12-30,2022
67,68,Son Of A Sinner,Jelly Roll,2022-12-30,2022
68,69,Wait In The Truck,HARDY Featuring Lainey Wilson,2022-12-30,2022


In [16]:
all_songs["Artist"][all_songs['Artist'] == "Jackson 5"] = "The Jackson 5"
all_songs["Artist"][all_songs['Artist'] == "Beatles"] = "The Beatles"

In [18]:
api = genius.Genius(GENIOUS_ACCESS_TOKEN, sleep_time=0.01, verbose=False)

In [19]:
all_song_data = pd.DataFrame()
start_time = datetime.now()
print("Started at {}".format(start_time))
for i in range(0, len(all_songs)):
    rolling_pct = int((i / len(all_songs)) * 100)
    print(str(rolling_pct) + "% complete." + " Collecting Record " + str(i) + " of " +
          str(len(all_songs)) + ". Year " + str(all_songs.iloc[i]['Year']) + "." + " Currently collecting " +
          all_songs.iloc[i]['Song Title'] + " by " + all_songs.iloc[i]['Artist'] + " " * 50, end="\r")
    song_title = all_songs.iloc[i]['Song Title']
    song_title = re.sub(" and ", " & ", song_title)
    artist_name = all_songs.iloc[i]['Artist']
    artist_name = re.sub(" and ", " & ", artist_name)

    try:
        song = api.search_song(song_title, artist=artist_name)
        #print(song)
        song_album = song.album
        song_album_url = song.album_url
        featured_artists = song.primary_artist
        song_lyrics = re.sub("\n", " ", song.lyrics)  #Remove newline breaks, we won't need them.
        song_media = song.media
        song_url = song.url
        # song_writer_artists = song.writer_artists
        song_year = song.year
    except:
        song_album = "null"
        song_album_url = "null"
        featured_artists = "null"
        song_lyrics = "null"
        song_media = "null"
        song_url = "null"
        # song_writer_artists = "null"
        song_year = "null"

    row = {
        "Year": all_songs.iloc[i]['Year'],
        "Rank": all_songs.iloc[i]['Rank'],
        "Song Title": all_songs.iloc[i]['Song Title'],
        "Artist": all_songs.iloc[i]['Artist'],
        "Album": song_album,
        "Album URL": song_album_url,
        "Featured Artists": featured_artists,
        "Lyrics": song_lyrics,
        "Media": song_media,
        "Song URL": song_url,
        # "Writers": song_writer_artists,
        "Release Date": song_year
    }
    all_song_data = all_song_data.append(row, ignore_index=True)
end_time = datetime.now()
print("\nCompleted at {}".format(start_time))
print("Total time to collect: {}".format(end_time - start_time))


Started at 2023-03-10 16:30:35.658183
Record 3 of 200. Year 2022. Currently collecting A Holly Jolly Christmas by Burl Ives                                                  2% complete. Collecting Record 5 of 200. Year 2022. Currently collecting It's The Most Wonderful Time Of The Year by Andy Williams                                                  3% complete. Collecting Record 7 of 200. Year 2022. Currently collecting Feliz Navidad by Jose Feliciano                                                  4% complete. Collecting Record 9 of 200. Year 2022. Currently collecting Unholy by Sam Smith & Kim Petras                                                  5% complete. Collecting Record 11 of 200. Year 2022. Currently collecting Santa Tell Me by Ariana Grande                                                  6% complete. Collecting Record 13 of 200. Year 2022. Currently collecting Underneath The Tree by Kelly Clarkson                                                  7% complete. Collecting

In [35]:
song = api.search_song('Hold Me Closer', artist='Elton John')

In [40]:
# for i in song.to_dict():
print(api.song_annotations(song.id))
    # print(api.song_comments(song.to_dict()['id']))

[('Hold me closer, tiny dancer', [['Eventually, it becomes apparent that they are not singing about actor Tony Danza, unless Elton and Britney want him to hold them closer.  Which could happen.\n\nEdit: I thought I made it clear, this isn’t about Tony Danza, despite the popular mondegreen.']])]


In [21]:
all_song_data.tail(5)

,Year,Rank,Song Title,Artist,Album,Album URL,Featured Artists,Lyrics,Media,Song URL,Writers,Release Date
195,2022,96,Down Home,Jimmie Allen,null,null,null,null,null,null,null,null
196,2022,97,One Thing At A Time,Morgan Wallen,null,null,null,null,null,null,null,null
197,2022,98,Hold Me Closer,Elton John & Britney Spears,null,null,null,null,null,null,null,null
198,2022,99,Far,SZA,null,null,null,null,null,null,null,null
199,2022,100,All Mine,Brent Faiyaz,null,null,null,null,null,null,null,null


In [ ]:
all_song_data.to_csv("all_songs_data.csv")
all_song_data.to_json("all_song_data.json", orient='records')

In [ ]:
loaded_song_dataset = pd.read_csv("all_songs_data.csv", index_col=0)

In [ ]:
display(loaded_song_dataset.head())
display(loaded_song_dataset.tail())

In [ ]:
songs_with_lyrics_dataset = loaded_song_dataset.dropna(subset=['Lyrics'])


## Understanding lyrics with T5 transformer

* TODO: ADD BLEU/ROUGE evaluation

In [ ]:
MODEL_NAME = 'igorktech/t5-base-en'
SAVE_MODEL_NAME = 't5-base-en-explainer'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device);

BATCH_SIZE = 32
REPORT_STEPS = 1000
EPOCHS = 1
LEARNING_RATE = 1e-4
TASK_PREFIX = "explain | "
MAX_INPUT = 64
MAX_OUTPUT = 64
# model.config.max_length = MAX_OUTPUT #not mandatory
SAVE_TO_HUB = True

In [ ]:
pairs = []
for a, b in zip(data['train']['short'], data['train']['long']):
    pairs.append([TASK_PREFIX + a, b])

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
model.train()
losses = []
for epoch in range(EPOCHS):
    print('EPOCH', epoch)
    random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / BATCH_SIZE)):
        batch = pairs[i * BATCH_SIZE: (i + 1) * BATCH_SIZE]
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding="longest", \
                      max_length=MAX_INPUT, truncation=True).to(model.device)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding="longest", \
                      max_length=MAX_OUTPUT, truncation=True, ).to(model.device)
        y.input_ids[y.input_ids == 0] = -100
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        if i % REPORT_STEPS == 0:
            print('step', i, 'loss', np.mean(losses[-REPORT_STEPS:]))

### Save model

In [ ]:
if SAVE_TO_HUB:
    notebook_login()
    model.push_to_hub(SAVE_MODEL_NAME)
    tokenizer.push_to_hub(SAVE_MODEL_NAME)
else:
    model.save_pretrained(SAVE_MODEL_NAME)
    tokenizer.save_pretrained(SAVE_MODEL_NAME)

### Testing model

In [ ]:
model.eval()


def answer(x, **kwargs):
    inputs = tokenizer(x, return_tensors='pt').to(model.device)
    with torch.no_grad():
        hypotheses = model.generate(**inputs, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [ ]:
print('model: ', answer(TASK_PREFIX + 'What is love?'))